# Evaluate SVI by linear models

For example, ARIMA(1, 1, 0) is employed as a base model:

$$
\Delta \ln \mathrm{IC}_{t} = \beta_{0} + \beta_{1} \Delta \ln \mathrm{IC}_{t-1} + \varepsilon_{t}.
$$

As evaluation model, we build ARIMAX(1, 1, 0) with exogenous:

$$
\Delta \ln \mathrm{IC}_{t} = \beta_{0} + \beta_{1} \Delta \ln \mathrm{IC}_{t-1} + \Delta \mathrm{SVI}_{t-1} + \varepsilon_{t},
$$

then compare the two MAPE:

$$
\mathrm{MAPE} = \frac{1}{T} \sum_{t = 1}^T \left\lvert 1 -\frac{\widehat{\Delta \ln \mathrm{IC}_{t}}}{\Delta \ln \mathrm{IC}_{t}} \right\rvert
$$

In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import datetime
import pickle
import sys
import warnings
from pathlib import Path
from pprint import pprint

from statsmodels.tools.sm_exceptions import ConvergenceWarning

warnings.simplefilter('ignore', ConvergenceWarning)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dateutil.relativedelta import relativedelta
from sktime.forecasting.arima import ARIMA
from statsmodels.tsa.seasonal import STL

sys.path.append("../../src")
from ftrends.construction import RBC
from stattools.test import DieboldMariano

import data

In [3]:
# endogenous
file = Path("../../data/raw/macro/ICSA.csv")
df_endog = pd.read_csv(file, index_col=0)
df_endog.index = pd.to_datetime(df_endog.index)

# exogenous
file = Path("../../data/raw/google/weekly/unemployment-office/svi_cat0.pkl")
with file.open(mode="rb") as f:
    dfs = pickle.load(f)
rbc = RBC(dfs)
df_exog = rbc.knit(direction="forward")
df_exog.index = pd.to_datetime(df_exog.index).map(lambda x: x + relativedelta(days=6))


endog, exog = "IC", "SVI"
df = pd.concat([df_endog, df_exog], axis=1) # Concatenate
df.dropna(how='any', inplace=True) # Drop NaN
df.rename(columns = dict(zip(df.columns, [endog, exog])), inplace=True) # Rename

seasonal adjustment, drop COVID

In [4]:
def seasonal_adjust(df:pd.DataFrame, factor:str, **stl_kwargs:dict):
    res = STL(df[factor], **stl_kwargs).fit()
    df["SVI"] = res.trend

def remove_covid(df:pd.DataFrame):
    covid_start = datetime.datetime(2020, 1, 1)
    return df[df.index < covid_start]

## Evaluation by in-sample data

endogenous only

In [5]:
_df = df.copy()
_df = remove_covid(_df) # Remove COVID-19
_df[endog] = np.log(_df[endog]).diff(1) # logarithm ratio
_df.dropna(how="any", inplace=True)

model = ARIMA(order=(1,0,0), seasonal_order=(0,0,0,0), maxiter=10000)
forecaster = model.fit(_df[endog])

print(model.summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  833
Model:               SARIMAX(1, 0, 0)   Log Likelihood                1466.701
Date:                Tue, 18 Oct 2022   AIC                          -2927.401
Time:                        04:48:47   BIC                          -2913.226
Sample:                    01-17-2004   HQIC                         -2921.966
                         - 12-28-2019                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0006      0.001     -0.408      0.684      -0.003       0.002
ar.L1         -0.1933      0.034     -5.707      0.000      -0.260      -0.127
sigma2         0.0017   5.36e-05     32.289      0.0

w/ exogenous

In [6]:
_df = df.copy()
_df = remove_covid(_df) # Remove COVID-19
seasonal_adjust(_df, exog) # Seasonal adjusted
_df[endog] = np.log(_df[endog]).diff(1) # logarithm ratio
_df[exog] = _df[exog].diff(1).shift(1) # difference and shift t -> t-1
_df.dropna(how="any", inplace=True)

model = ARIMA(order=(1,0,0), seasonal_order=(0,0,0,0), maxiter=10000)
forecaster = model.fit(_df[endog], _df[exog])

print(model.summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  832
Model:               SARIMAX(1, 0, 0)   Log Likelihood                1468.129
Date:                Tue, 18 Oct 2022   AIC                          -2928.259
Time:                        04:48:47   BIC                          -2909.363
Sample:                    01-24-2004   HQIC                         -2921.013
                         - 12-28-2019                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0005      0.001     -0.354      0.723      -0.003       0.002
SVI            0.0511      0.021      2.449      0.014       0.010       0.092
ar.L1         -0.2000      0.034     -5.949      0.0

## Evaluation by out-sample data

ARIMA(1, 1, 0) (base)

In [7]:
_df = df.copy()
_df = remove_covid(_df) # Remove COVID-19
_df[endog] = np.log(_df[endog]).diff(1) # logarithm ratio
_df.dropna(how="any", inplace=True)

model = ARIMA(order=(1,0,0), seasonal_order=(0,0,0,0), maxiter=10000)
forecaster = model.fit(_df[endog])

# To save
list_df_pred = []

# model parameter
window=52
step=1

for i in range(len(_df)):
    train_start_date = _df.index.min() + relativedelta(weeks=step*i)
    train_end_date = train_start_date + relativedelta(weeks=window-1)
    test_start_date = train_end_date + relativedelta(weeks=1)
    test_end_date = train_end_date + relativedelta(weeks=step)

    if test_end_date > _df.index.max():
        break

    df_train = _df[(_df.index >= train_start_date) & (_df.index <= train_end_date)]
    df_test = _df[(_df.index >= test_start_date) & (_df.index <= test_end_date)]

    model = ARIMA(order=(1,0,0), seasonal_order=(0,0,0,0), maxiter=100000000)
    forecaster = model.fit(df_train[endog])
    df_pred = forecaster.predict(fh=pd.DatetimeIndex(df_test.index, freq='W-SAT'))
    
    list_df_pred.append(df_pred)

df_pred_base = pd.concat(list_df_pred).to_frame(name='pred')

ARIMAX(1, 1, 0)

In [8]:
_df = df.copy()
_df = remove_covid(_df) # Remove COVID-19
seasonal_adjust(_df, exog) # Seasonal adjusted
_df[endog] = np.log(_df[endog]).diff(1) # logarithm ratio
_df[exog] = _df[exog].diff(1).shift(1) # difference and shift t -> t-1
_df.dropna(how="any", inplace=True)

_df = df.copy()
_df = remove_covid(_df) # Remove COVID-19
_df[endog] = np.log(_df[endog]).diff(1) # logarithm ratio
_df.dropna(how="any", inplace=True)

model = ARIMA(order=(1,0,0), seasonal_order=(0,0,0,0), maxiter=10000)
forecaster = model.fit(_df[endog])

# To save
list_df_pred = []

# model parameter
window=52
step=1

for i in range(len(_df)):
    train_start_date = _df.index.min() + relativedelta(weeks=step*i)
    train_end_date = train_start_date + relativedelta(weeks=window-1)
    test_start_date = train_end_date + relativedelta(weeks=1)
    test_end_date = train_end_date + relativedelta(weeks=step)

    if test_end_date > _df.index.max():
        break

    df_train = _df[(_df.index >= train_start_date) & (_df.index <= train_end_date)]
    df_test = _df[(_df.index >= test_start_date) & (_df.index <= test_end_date)]

    model = ARIMA(order=(1,0,0), seasonal_order=(0,0,0,0), maxiter=100000000)
    forecaster = model.fit(df_train[endog], df_train[exog])
    df_pred = forecaster.predict(X=df_test[exog], fh=pd.DatetimeIndex(df_test.index, freq='W-SAT'))
    
    list_df_pred.append(df_pred)

df_pred_model = pd.concat(list_df_pred).to_frame(name='pred')

Summary

In [9]:
df_summary = pd.concat(
    [
        _df.loc[:, [endog]].rename(columns={endog:'target'}),
        df_pred_base.rename(columns={'pred':'pred_base'}),
        df_pred_model
    ],
    axis=1
)
df_summary.index = pd.to_datetime(df_summary.index)

In [10]:
_df = df_summary.dropna()
pe_base = np.abs(_df["pred_base"] - _df["target"])
pe = np.abs(_df["pred"] - _df["target"])
de = pe_base - pe
de_ma = de.rolling(52).mean().dropna()

trace = go.Bar(
    x=de_ma.index,
    y=de_ma*100,
    name=rf'Difference in forecast errors (M(0, 0) - M(1, 0))',
    yaxis='y'
)

layout = go.Layout(
    width=1500,
    height=800,
    font_size=15,
    hoverlabel_font_size=20,
    legend={"x":0.99, "y":1.1, "xanchor":"right", "yanchor":"bottom","orientation":"h"},
    xaxis={"title":"Date"},
    yaxis={"title":"Difference in forecast errors [%]", "side":"left"},
    yaxis2={"title": "Logarithm of Initial Claims", "side":"right", "overlaying":"y"},
    barmode="stack",
    hovermode="x",
    template="plotly_dark"
)
fig = go.Figure(data=[trace], layout=layout)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

DM test

In [11]:
_df = df_summary.dropna()
target, pred0, pred1 = _df["target"], _df["pred_base"], _df["pred"]
ret = DieboldMariano(target, pred0, pred1, criterion="MAE")

pprint(ret)

{'DM-statistic': -0.36755176375478743,
 'p-value': 0.7133071939969811,
 'result': 'The 2nd prediction is not said to be higher accuracy than the 1st '
           'one under 5% significance.'}
